In [5]:
import pandas as pd
import numpy as np
import requests
import csv
import folium
import ast

In [ ]:
# Load the data
df = pd.read_excel('MergedData.xlsx')

# List of columns that contain hospital names
columns = ['Providers_08_09', 'Providers_09_10', 'Providers_10_11', 'Providers_11_12', 
           'Providers_12_13', 'Providers_13_14', 'Providers_14_15', 'Providers_15_16', 
           'Providers_16_17', 'Providers_17_18', 'Providers_18_19', 'Providers_19_20', 
           'Providers_20_21', 'Providers_21_22']

# Collect all unique hospital names
hospitalsList = pd.unique(df[columns].values.ravel('K'))

# Print the list of unique hospital names
print(hospitalsList)


In [ ]:
# Remove nan values from the list
hospitalsList = [hospital for hospital in hospitalsList if not pd.isnull(hospital)]

In [ ]:
print(len(hospitalsList))

In [ ]:
hospitalsList

In [ ]:
# Create a new list with only those names that contain 'Hospital', case-insensitive
hospitals_with_word = [name for name in hospitalsList if isinstance(name, str) and 'hospital' in name.lower()]

# Print the new list
print(hospitals_with_word)

In [ ]:
print(len(hospitals_with_word))

In [ ]:
hospitals = hospitalsList

In [ ]:
def search_place(api_key, query):
    url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={query}&inputtype=textquery&fields=photos,formatted_address,name,rating,opening_hours,geometry&key={api_key}&language=en"
    response = requests.get(url)
    data = response.json()
    if 'candidates' in data and len(data['candidates']) > 0:
        place = data['candidates'][0]
        return place
    else:
        return None

api_key = 'API Key'

# Open (or create) a CSV file for writing
with open('hospital_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['name', 'address', 'coordinates']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header row
    writer.writeheader()

    not_found_counter = 0  # Counter for places not found
    
    for hospital in hospitals:
        query = hospital
        try:
            place = search_place(api_key, query)
            if place:
                print(f"Place: {place['name']}")
                print(f"Address: {place['formatted_address']}")
                print(f"Coordinates: {place['geometry']['location']}")

                # Write the place data to the CSV file
                writer.writerow({'name': place['name'], 'address': place['formatted_address'], 'coordinates': place['geometry']['location']})
            else:
                not_found_counter += 1
                print(f"Could not find place for {query}. Total not found: {not_found_counter}")
        except UnicodeEncodeError:
            print(f"UnicodeEncodeError occurred for {query}. Skipping this entry.")


In [ ]:
not_found_counter

Could nopt find total 477 instances 

In [10]:
# Load the CSV file
df = pd.read_csv('hospital_data.csv')

# Convert 'coordinates' from string to dictionary
df['coordinates'] = df['coordinates'].apply(ast.literal_eval)

# Extract 'lat' and 'lng' into separate columns
df['latitude'] = df['coordinates'].apply(lambda x: x['lat'])
df['longitude'] = df['coordinates'].apply(lambda x: x['lng'])

# Select the first 50 rows

# Create a map centered around the first coordinate
m = folium.Map(location=[df['latitude'].iloc[0], df['longitude'].iloc[0]], zoom_start=13)

# Add points to the map
for index, row in df.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup=row['name']).add_to(m)

# Display the map
m

In [13]:
# Create a new DataFrame with addresses not in the United Kingdom
df_not_uk = df[~df['address'].str.contains('United Kingdom')]

# Select the 'name' and 'address' columns
df_not_uk = df_not_uk[['name', 'address']]

# Convert the DataFrame to a list of dictionaries
names_addresses_not_in_uk = df_not_uk.to_dict('records')

In [14]:
names_addresses_not_in_uk

[{'name': 'Kensington and Chelsea',
  'address': 'Kensington and Chelsea, London SW7 4DG, UK'},
 {'name': 'Berkshire East Mountain Resort',
  'address': '66 Thunder Mountain Rd, Charlemont, MA 01339, United States'},
 {'name': 'St. Marien-Hospital Berlin',
  'address': 'Gallwitzallee 123 – 143, 12249 Berlin, Germany'},
 {'name': 'St. Catherine Hospital',
  'address': 'Seckbacher Landstraße 65, 60389 Frankfurt am Main, Germany'},
 {'name': 'Milton District Hospital',
  'address': '725 Bronte St. S, Milton, ON L9T 9K1, Canada'},
 {'name': 'The George Eliot Hospital N H S Trust',
  'address': 'The George Eliot Hospital N H S Trust, College St, Nuneaton CV10 7DJ, UK'},
 {'name': 'BMI The Kings Oak Hospital',
  'address': 'BMI The Kings Oak Hospital, Enfield EN2 8SD, UK'},
 {'name': 'Essex', 'address': 'Essex, UK'},
 {'name': 'Berkshire East Mountain Resort',
  'address': '66 Thunder Mountain Rd, Charlemont, MA 01339, United States'},
 {'name': 'Lazarus Hospiz',
  'address': 'Bernauer Str. 